In [1]:
import pandas as pd

import glob

path='c:\\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia'

files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]

for f in files:
    print(f)

c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_1000000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_1000000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_1400000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_1400000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_400000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1000000_IF_400000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1400000_IF_1000000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\2DCONV_H_A15_HF_1400000_IF_1000000_MF

c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_2000000_IF_400000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_1000000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_1000000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_1400000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_1400000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_400000_MF_600000_dev_0_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A15_HF_400000_IF_400000_MF_600000_dev_1_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench\Logs Idle de Energia\SYRK_H_A7_HF_1000000_IF_1000000_MF_600000_dev_0_log_energi

In [7]:
def group_results(file,name):
    df_temp = pd.DataFrame()
    dict_temp = {}    

    for line in file:
        if ":" in line:
            if line[0] == 'm':
                break
                
            #print(line.split(':')[1].replace('s',''))
            key, value = line.split(':')[0], float(line.split(':')[1].replace('s',''))
            dict_temp[key] =  float(value)
            #print(key,':',value)
        else: 
            if 'm' in line:
                key = line.split('\t')[0]
                mi = int(line.split('\t')[1].split('m')[0])
                se = float(line.split('\t')[1].split('m')[1].replace('s',''))
                value = mi*60 + se
                dict_temp[key] =  float(value)
                #print(key,':',value)
                if "sys" in line:
                    df_temp = df_temp.append(dict_temp, ignore_index=True)
                    dict_temp = dict.fromkeys(dict_temp, 0)
    
    df_temp = df_temp.mean().to_dict()
    
    app = name.split('\\')[-1].split('_')[0]
    Host = name.split('_')[2]
    Host_freq = int(name.split('_')[4])
    idle1_freq = int(name.split('_')[6])
    if Host == 'A15':
        idle1_dev = 'A7'
    
    if Host == 'A7':
        idle1_dev = 'A15'
    
    if(int(name.split('_')[10]) == 0):
        dev = 'GPU'
        dev_freq = 600000000.0
        idle1_dev = name.split('_')[2]
        idle1_freq = int(name.split('_')[4])
        if idle1_dev == 'A15':
            idle2_dev = 'A7'
    
        if idle1_dev == 'A7':
            idle2_dev = 'A15'
    
        idle2_freq = int(name.split('_')[6])
    else:
        dev = Host
        dev_freq = Host_freq
        idle2_dev = 'GPU'
        idle2_freq = 600000000.0
        
    df_temp['App'] = app
    df_temp['Host'] = Host
    #df_temp['Host_freq'] = Host_freq
    df_temp['Freq'] = dev_freq
    df_temp['Dev'] = str(dev)
    df_temp['Idle1'] = idle1_dev
    df_temp['Idle1_Freq'] = idle1_freq
    df_temp['Idle2'] = idle2_dev
    df_temp['Idle2_Freq'] = idle2_freq
    
    return df_temp

In [8]:
df = pd.DataFrame()

for path in files:
    log_data=open(path,'r')
    temp_dict = group_results(log_data, path)
    df = df.append(temp_dict, ignore_index=True)
    col = df['Freq']
    df.drop(labels=['Freq'], axis=1,inplace = True)
    df.insert(0, 'Freq', col)
    col = df['Dev']
    df.drop(labels=['Dev'], axis=1,inplace = True)
    df.insert(0, 'Dev', col)
    col = df['Host']
    df.drop(labels=['Host'], axis=1,inplace = True)
    df.insert(0, 'Host', col)
    col = df['App']
    df.drop(labels=['App'], axis=1,inplace = True)
    df.insert(0, 'App', col)


In [9]:
df.head(49)

,App,Host,Dev,Freq,A15_Acumulada,A7_Acumulada,GPU_Acumulada,Idle1,Idle1_Freq,Idle2,Idle2_Freq,MEM_Acumulada,real,sys,user
0,2DCONV,A15,GPU,600000000.0,3.359609,0.393837,0.413309,A15,1000000.0,A7,1000000.0,0.543915,6.0376,1.1083,4.7077
1,2DCONV,A15,A15,1000000.0,4.311673,0.423279,0.404772,A7,1000000.0,GPU,600000000.0,0.660535,6.1519,1.0010,5.7060
2,2DCONV,A15,GPU,600000000.0,3.257579,0.727040,0.417100,A15,1000000.0,A7,1400000.0,0.515334,6.0206,1.0839,4.7250
3,2DCONV,A15,A15,1000000.0,4.421417,0.821155,0.404862,A7,1400000.0,GPU,600000000.0,0.653591,6.1184,0.9802,5.7242
4,2DCONV,A15,GPU,600000000.0,3.651329,0.156945,0.549125,A15,1000000.0,A7,400000.0,0.608304,6.1716,1.1871,4.7486
5,2DCONV,A15,A15,1000000.0,4.454369,0.171045,0.405789,A7,400000.0,GPU,600000000.0,0.647026,6.2451,1.0800,5.7360
6,2DCONV,A15,GPU,600000000.0,4.316624,0.306199,0.464835,A15,1400000.0,A7,1000000.0,0.429349,4.9852,0.9270,3.8477
7,2DCONV,A15,A15,1400000.0,4.866823,0.301052,0.289337,A7,1000000.0,GPU,600000000.0,0.453167,4.9894,0.8058,4.6228
8,2DCONV,A15,GPU,600000000.0,4.431020,0.589120,0.411210,A15,1400000.0,A7,1400000.0,0.425060,4.9797,0.9236,3.8483
9,2DCONV,A15,A15,1400000.0,4.955639,0.586794,0.288421,A7,1400000.0,GPU,600000000.0,0.456443,4.9815,0.8027,4.6115


In [5]:
to_mantain = ['Dev','Freq','Idle1','Idle1_Freq','Idle2','Idle2_Freq']
df_configs = df[df['App'] == '2DCONV']
df_configs = df_configs.loc[:,df_configs.columns.isin(to_mantain)]
df_configs

,Dev,Freq,Idle1,Idle1_Freq,Idle2,Idle2_Freq
0,GPU,600000000.0,A15,1000000.0,A7,1000000.0
1,A15,1000000.0,A7,1000000.0,GPU,600000000.0
2,GPU,600000000.0,A15,1000000.0,A7,1400000.0
3,A15,1000000.0,A7,1400000.0,GPU,600000000.0
4,GPU,600000000.0,A15,1000000.0,A7,400000.0
5,A15,1000000.0,A7,400000.0,GPU,600000000.0
6,GPU,600000000.0,A15,1400000.0,A7,1000000.0
7,A15,1400000.0,A7,1000000.0,GPU,600000000.0
8,GPU,600000000.0,A15,1400000.0,A7,1400000.0
9,A15,1400000.0,A7,1400000.0,GPU,600000000.0
